In [15]:
from flask import Flask, render_template, send_from_directory, request, Response, stream_with_context
import subprocess

app = Flask(__name__, template_folder='C:/Users/leramero/Desktop/Portfolio', static_folder='C:/Users/leramero/Desktop/Portfolio/assets')

class PokemonCombat:
    def __init__(self):
        self.proceso = None

    def iniciar_combate(self):
        # Iniciar la ejecución del script
        script = "projects/codes/pokemon.py"
        self.proceso = subprocess.Popen(["python", "-u", script], stdout=subprocess.PIPE, universal_newlines=True)
        return Response("Iniciado", mimetype="text/event-stream", headers={
            "Cache-Control": "no-cache",
            "Content-Type": "text/event-stream"
        })

    def combat_stream(self):
        # Leer la salida del script y enviarla como eventos
        def generar_eventos():
            while True:
                salida = self.proceso.stdout.readline()
                if salida:
                    yield "data: " + salida + "\n\n"
                else:
                    self.cerrar_proceso()
                    break
        return Response(generar_eventos(), mimetype="text/event-stream", headers={
            "Cache-Control": "no-cache",
            "Content-Type": "text/event-stream"
        })

    def cerrar_proceso(self):
        # Cerrar el proceso
        if self.proceso:
            self.proceso.kill()
            self.proceso = None

pokemon_combat = PokemonCombat()

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/assets/<path:path>")
def send_static(path):
    return send_from_directory("assets", path)

@app.route("/projects/assets/pk/<path:path>")
def send_static_pk(path):
    return send_from_directory("projects/assets/pk", path)

@app.route("/projects/assets/<path:path>")
def send_static_pr(path):
    return send_from_directory("projects/assets", path)

@app.route("/assets/images/<path:path>")
def send_image(path):
    return send_from_directory("assets/images", path)

@app.route("/projects/assets/images/<path:path>")
def send_image_pk(path):
    return send_from_directory("projects/assets/images", path)

@app.route("/projects/<path:path>")
def pokemon(path):
    return render_template("projects/pokemon.html")

@app.route("/pokemon_combat", methods=["POST"])
def pokemon_combat_inicio():
    return pokemon_combat.iniciar_combate()

@app.route("/pokemon_combat_stream")
def pokemon_combat_stream():
    return pokemon_combat.combat_stream()

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Aug/2024 16:16:44] "GET /projects/pokemon.html HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:44] "GET /projects/assets/css-pokemon-gameboy.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:44] "GET /projects/pokemon.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:44] "GET /projects/assets/images/fondo_pk.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:44] "GET /projects/assets/images/pk.ico HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:45] "POST /pokemon_combat HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2024 16:16:45] "GET /favicon.ico HTTP/1.1" 404 -
